In [1]:
import duckdb

In [7]:
# Conectando ao banco de dados "dados_duckdb" com permissão total (leitura e escrita).

con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [10]:
# Realizando uma consulta (SELECT *) na tabela 'produtos' e convertendo o resultado em um DataFrame do Pandas.

df = con.execute("SELECT * FROM produtos").fetchdf()
df.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10003,PREGO,BT10,100,60.0
3,10005,MACHADO,BT50,100,100.0
4,10002,MARTELO,BT50,100,1500.0


In [ ]:
# Removendo as colunas especificadas do DataFrame 'df' e salvando o resultado em 'df2'.

df2 = df.drop(columns=['id_categoria', 'id_fornecedor'])
df2.head(10)

,id,nm_produto,vl_preco
0,10001,PARAFUSO,100.0
1,10004,SERRA,200.0
2,10003,PREGO,60.0
3,10005,MACHADO,100.0
4,10002,MARTELO,1500.0


In [ ]:
# Criando a tabela 'dim_produtos' (se ela não existir) e definindo as colunas.

con.execute("""
    CREATE TABLE IF NOT EXISTS dim_produtos (
        id_produto BIGINT,
        NM_PRODUTO TEXT,
        VL_PRODUTO FLOAT
            )
""")

In [15]:
# Verificando a tabela criada (sem dados ainda)

df_dim = con.execute("SELECT * FROM dim_produtos").fetchdf()
df_dim.head(10)

,id_produto,NM_PRODUTO,VL_PRODUTO


In [17]:
# Alterando a tabela, deixando as colunas com escrita em minúsculo

# Renomeando a coluna NM_PRODUTO para nm_produto.
con.execute("""
    ALTER TABLE dim_produtos RENAME COLUMN NM_PRODUTO TO nm_produto
""")

# Renomeando a coluna VL_PRODUTO para vl_produto.
con.execute("""
    ALTER TABLE dim_produtos RENAME COLUMN VL_PRODUTO TO vl_produto
""")





In [20]:
# Conferindo a alteração com o comando PRAGMA:

con.execute("PRAGMA table_info('dim_produtos')").fetchall()

# O resultado terá as seguinte estrutura:  
# Índice (cid),	Nome da Coluna,	Tipo de Dados,	Not Null?,	Valor Padrão,	PK (Chave Primária)?

[(0, 'id_produto', 'BIGINT', False, None, False),
 (1, 'nm_produto', 'VARCHAR', False, None, False),
 (2, 'vl_produto', 'FLOAT', False, None, False)]

In [21]:
# Outra forma de conferir é verificar a tabela criada (sem dados ainda)

df_dim = con.execute("SELECT * FROM dim_produtos").fetchdf()
df_dim.head(10)

,id_produto,nm_produto,vl_produto


In [22]:
# Populando a tabela dim_produtos com os dados salvos em df2

con.execute("INSERT INTO dim_produtos SELECT * FROM df2")

In [23]:
# Verificando a tabela com os dados inseridos

df_dim = con.execute("SELECT * FROM dim_produtos").fetchdf()
df_dim.head(10)

,id_produto,nm_produto,vl_produto
0,10001,PARAFUSO,100.0
1,10004,SERRA,200.0
2,10003,PREGO,60.0
3,10005,MACHADO,100.0
4,10002,MARTELO,1500.0


In [24]:
con.close()